In [ ]:
from collections import defaultdict
import json
import os
import random
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import torch
from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from research.weight_estimation.old.weight_estimator_old import NormalizeCentered2D, NormalizedStabilityTransform, ToTensor, Network
from research.weight_estimation.keypoint_utils.optics import pixel2world
from scipy.spatial.distance import pdist

pd.set_option('display.max_rows', 500)

In [ ]:
f = '/root/data/alok/biomass_estimation/playground/result_bolaks_pen_id_88_2020-02-10_2020-03-10.csv'
df = pd.read_csv(f)

In [ ]:
# initialize data transforms so that we can run inference with neural network
normalize_centered_2D_transform = NormalizeCentered2D(center=False)
normalized_stability_transform = NormalizedStabilityTransform()
to_tensor_transform = ToTensor()

# Get neural network weights from sample training

model_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2019-11-08T00-13-09/nn_epoch_798.pb'
model_f, _, _ = s3_access_utils.download_from_url(model_url)
torch.load(model_f)

weight_predictions = []
for idx, row in df.head(1).iterrows():
    keypoints = json.loads(row.annotation.replace("'", '"'))
    cm = json.loads(row.camera_metadata.replace("'", '"'))
    input_sample = {
        'keypoints': keypoints,
        'cm': cm,
        'stereo_pair_id': row.id,
        'single_point_inference': True
    }
    nomralized_centered_2D_kps = \
        normalize_centered_2D_transform.__call__(input_sample)

    normalized_stability_kps = normalized_stability_transform.__call__(nomralized_centered_2D_kps)
    tensorized_kps = to_tensor_transform.__call__(normalized_stability_kps)
    weight_prediction = network(tensorized_kps['kp_input']).item() * 1e4
    weight_predictions.append(weight_prediction)


In [ ]:
weight_predictions

In [ ]:
df.estimated_weight_g.head(10)

In [ ]:
keypoints

In [ ]:
cm

In [ ]:
tensorized_kps['kp_input'].numpy()

In [ ]:
nomralized_centered_2D_kps

In [ ]:
normalized_stability_kps

In [ ]:
class NormalizedStabilityTransform(object):
    """
        Transforms world keypoints into a more stable coordinate system - this will lead to better
        training / convergence
    """
    
    def __call__(self, sample):
        modified_kps, label, stereo_pair_id, cm = \
            sample['modified_kps'], sample['label'], sample['stereo_pair_id'], sample['cm']
        modified_wkps = pixel2world(modified_kps['leftCrop'], modified_kps['rightCrop'], cm)
        print(modified_wkps)
        stabilized_coordinates = {}
        for bp in BODY_PARTS:
            wkp = modified_wkps[bp]
            stabilized_kp_info = [0.5 * wkp[0]/wkp[1], 0.5 * wkp[2]/wkp[1], 0.5 * 0.1/wkp[1]]
            stabilized_coordinates[bp] = stabilized_kp_info
            
        normalized_label = label * 1e-4 if label else None
        
        transformed_sample = {
            'kp_input': stabilized_coordinates,
            'label': normalized_label,
            'stereo_pair_id': stereo_pair_id,
            'single_point_inference': sample.get('single_point_inference')
        }
        
        return transformed_sample

import numpy as np


def euclidean_distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))


def convert_to_world_point(x, y, d, parameters):
    """ from pixel coordinates to world coordinates """
    # get relevant parameters
    pixel_count_width = parameters["pixelCountWidth"]
    pixel_count_height = parameters["pixelCountHeight"]
    sensor_width = parameters["imageSensorWidth"]
    sensor_height = parameters["imageSensorHeight"]
    focal_length = parameters["focalLength"]

    image_center_x = pixel_count_width / 2.0
    image_center_y = pixel_count_height / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y
    print(px_x, px_z)

    sensor_x = px_x * (sensor_width / pixel_count_width)
    sensor_z = px_z * (sensor_height / pixel_count_height)

    # now move to world coordinates
    world_y = d
    world_x = (world_y * sensor_x) / focal_length
    world_z = (world_y * sensor_z) / focal_length
    return np.array([world_x, world_y, world_z])


def depth_from_disp(disp, parameters):
    """ calculate the depth of the point based on the disparity value """
    focal_length_pixel = parameters["focalLengthPixel"]

    baseline = parameters["baseline"]
    depth = focal_length_pixel * baseline / np.array(disp)
    return depth


def pixel2world(left_crop, right_crop, parameters):
    """2D pixel coordinates to 3D world coordinates"""

    # first create a dic with crop keypoints
    image_coordinates = {"leftCrop": {},
                         "rightCrop": {}}
    for keypoint in left_crop:
        name = keypoint["keypointType"]
        image_coordinates["leftCrop"][name] = [keypoint["xFrame"], keypoint["yFrame"]]
    for keypoint in right_crop:
        name = keypoint["keypointType"]
        image_coordinates["rightCrop"][name] = [keypoint["xFrame"], keypoint["yFrame"]]

    # then loop through the right crop keypoints and calculate the world coordinates
    world_coordinates = {}
    for keypoint in left_crop:
        name = keypoint["keypointType"]
        disparity = image_coordinates["leftCrop"][name][0] - image_coordinates["rightCrop"][name][0]
        depth = depth_from_disp(disparity, parameters)
        world_point = convert_to_world_point(image_coordinates["leftCrop"][name][0],
                                             image_coordinates["leftCrop"][name][1],
                                             depth,
                                             parameters)
        world_coordinates[name] = world_point
    return world_coordinates


In [ ]:
import json, os

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from research.utils.data_access_utils import RDSAccessUtils



pd.set_option('display.max_rows', 500)



BODY_PARTS = sorted([
    'ADIPOSE_FIN',
    'ANAL_FIN',
    'DORSAL_FIN',
    'EYE',
    'PECTORAL_FIN',
    'PELVIC_FIN',
    'TAIL_NOTCH',
    'UPPER_LIP'
])



class NormalizeCentered2D(object):
    
    """
    Transforms the 2D left and right keypoints such that:
        (1) The center of the left image 2D keypoints is located at the center of the left image
            (i.e. 2D translation)
        (2) The left image keypoints are possibly flipped such that the upper-lip x-coordinate 
            is greater than the tail-notch coordinate. This is done to reduce the total number of 
            spatial orientations the network must learn from -> reduces the training size
        (3) The left image keypoints are then rotated such that upper-lip is located on the x-axis.
            As in (2), this is done to reduce the total number of spatial orientations the network 
            must learn from -> reduces the training size
        (4) Rescale all left image keypoints by some random number between 'lo' and 'hi' args
        (5) Apply Gaussian random noise "jitter" to each keypoint to mimic annotation error
        (5) For all transformations above, the right image keypoint coordinates are accordingly
            transformed such that the original disparity values are preserved for all keypoints
            (or adjusted during rescaling event)
    """


    def flip_center_kps(self, left_kps, right_kps):

        x_min_l = min([kp[0] for kp in left_kps.values()])
        x_max_l = max([kp[0] for kp in left_kps.values()])
        x_mid_l = np.mean([x_min_l, x_max_l])

        y_min_l = min([kp[1] for kp in left_kps.values()])
        y_max_l = max([kp[1] for kp in left_kps.values()])
        y_mid_l = np.mean([y_min_l, y_max_l])

        x_min_r = min([kp[0] for kp in right_kps.values()])
        x_max_r = max([kp[0] for kp in right_kps.values()])
        x_mid_r = np.mean([x_min_r, x_max_r])

        y_min_r = min([kp[1] for kp in right_kps.values()])
        y_max_r = max([kp[1] for kp in right_kps.values()])
        y_mid_r = np.mean([y_min_r, y_max_r])

        fc_left_kps, fc_right_kps = {}, {}
        flip_factor = 1 if left_kps['UPPER_LIP'][0] > left_kps['TAIL_NOTCH'][0] else -1
        for bp in BODY_PARTS:
            left_kp, right_kp = left_kps[bp], right_kps[bp]
            if flip_factor > 0:
                fc_left_kp = np.array([left_kp[0] - x_mid_l, left_kp[1] - y_mid_l])
                fc_right_kp = np.array([right_kp[0] - x_mid_l, right_kp[1] - y_mid_l])
            else:
                fc_right_kp = np.array([x_mid_r - left_kp[0], left_kp[1] - y_mid_r])
                fc_left_kp = np.array([x_mid_r - right_kp[0], right_kp[1] - y_mid_r])
            fc_left_kps[bp] = fc_left_kp
            fc_right_kps[bp] = fc_right_kp

        return fc_left_kps, fc_right_kps


    def _rotate_cc(self, p, theta):
        R = np.array([
            [np.cos(theta), -np.sin(theta)],
            [np.sin(theta), np.cos(theta)]
        ])

        rotated_kp = np.dot(R, p)
        return rotated_kp


    def rotate_kps(self, left_kps, right_kps):
        upper_lip_x, upper_lip_y = left_kps['UPPER_LIP']
        theta = np.arctan(upper_lip_y / upper_lip_x)
        r_left_kps, r_right_kps = {}, {}
        for bp in BODY_PARTS:
            rotated_kp = self._rotate_cc(left_kps[bp], -theta)
            r_left_kps[bp] = rotated_kp
            disp = abs(left_kps[bp][0] - right_kps[bp][0])
            r_right_kps[bp] = np.array([rotated_kp[0] - disp, rotated_kp[1]])

        return r_left_kps, r_right_kps


    def scale_kps(self, left_kps, right_kps, factor):
        s_left_kps, s_right_kps = {}, {}
        for bp in BODY_PARTS:
            left_kp, right_kp = left_kps[bp], right_kps[bp]
            s_left_kps[bp] = factor * np.array(left_kps[bp])
            s_right_kps[bp] = factor * np.array(right_kps[bp])

        return s_left_kps, s_right_kps


    def jitter_kps(self, left_kps, right_kps, jitter):
        j_left_kps, j_right_kps = {}, {}
        for bp in BODY_PARTS:
            j_left_kps[bp] = np.array([left_kps[bp][0] + np.random.normal(0, jitter), 
                                       left_kps[bp][1] + np.random.normal(0, jitter)])
            j_right_kps[bp] = np.array([right_kps[bp][0] + np.random.normal(0, jitter), 
                                        right_kps[bp][1] + np.random.normal(0, jitter)])

        return j_left_kps, j_right_kps



    def modify_kps(self, left_kps, right_kps, factor, jitter, cm, rotate=True, center=False):
        fc_left_kps, fc_right_kps = self.flip_center_kps(left_kps, right_kps)
        if rotate:
            r_left_kps, r_right_kps = self.rotate_kps(fc_left_kps, fc_right_kps)
            s_left_kps, s_right_kps = self.scale_kps(r_left_kps, r_right_kps, factor)
        else:
            s_left_kps, s_right_kps = self.scale_kps(fc_left_kps, fc_right_kps, factor)
        j_left_kps, j_right_kps  = self.jitter_kps(s_left_kps, s_right_kps, jitter)
        j_left_kps_list, j_right_kps_list = [], []
        if not center:
            for bp in BODY_PARTS:
                l_item = {
                    'keypointType': bp,
                    'xFrame': j_left_kps[bp][0] + cm['pixelCountWidth'] / 2.0,
                    'yFrame': j_left_kps[bp][1] + cm['pixelCountHeight'] / 2.0
                }

                r_item = {
                    'keypointType': bp,
                    'xFrame': j_right_kps[bp][0] + cm['pixelCountWidth'] / 2.0,
                    'yFrame': j_right_kps[bp][1] + cm['pixelCountHeight'] / 2.0
                }

                j_left_kps_list.append(l_item)
                j_right_kps_list.append(r_item)
        else:
            for bp in BODY_PARTS:
                l_item = {
                    'keypointType': bp,
                    'xFrame': j_left_kps[bp][0],
                    'yFrame': j_left_kps[bp][1]
                }

                r_item = {
                    'keypointType': bp,
                    'xFrame': j_right_kps[bp][0],
                    'yFrame': j_right_kps[bp][1]
                }

                j_left_kps_list.append(l_item)
                j_right_kps_list.append(r_item)


        modified_kps = {
            'leftCrop': j_left_kps_list,
            'rightCrop': j_right_kps_list
        }

        return modified_kps

    
    def __init__(self, lo=None, hi=None, jitter=0.0, rotate=True, center=False):
        self.lo = lo
        self.hi = hi
        self.jitter = jitter
        self.rotate = rotate
        self.center = center
    

    def __call__(self, sample):
        keypoints, cm, stereo_pair_id, label = \
            sample['keypoints'], sample['cm'], sample.get('stereo_pair_id'), sample.get('label')
        left_keypoints_list = keypoints['leftCrop']
        right_keypoints_list = keypoints['rightCrop']
        left_kps = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in left_keypoints_list}
        right_kps = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in right_keypoints_list}
        
        factor = 1.0 
        if self.lo and self.hi:
            factor = np.random.uniform(low=self.lo, high=self.hi)
            
        jitter = np.random.uniform(high=self.jitter)
        
        modified_kps = self.modify_kps(left_kps, right_kps, factor, jitter, cm, 
            rotate=self.rotate, center=self.center)

        kp_input = {}
        for idx, _ in enumerate(modified_kps['leftCrop']):
            left_item, right_item = modified_kps['leftCrop'][idx], modified_kps['rightCrop'][idx]
            bp = left_item['keypointType']
            kp_input[bp] = [left_item['xFrame'], left_item['yFrame'], right_item['xFrame'], right_item['yFrame']]


        transformed_sample = {
            'kp_input': kp_input,
            'modified_kps': modified_kps,
            'label': label,
            'stereo_pair_id': stereo_pair_id,
            'cm': cm,
            'single_point_inference': sample.get('single_point_inference')
        }
        
        return transformed_sample
        

class ToTensor(object):
    
    def __call__(self, sample):
        kp_input, label, stereo_pair_id = \
            sample['kp_input'], sample.get('label'), sample.get('stereo_pair_id')
        
        x = []
        for bp in BODY_PARTS:
            kp_data = kp_input[bp]
            x.append(kp_data)
        if sample.get('single_point_inference'):
            x = np.array([x])
        else:
            x = np.array(x)
        
        kp_input_tensor = torch.from_numpy(x).float()
        
        tensorized_sample = {
            'kp_input': kp_input_tensor
        }

        if label:
            label_tensor = torch.from_numpy(np.array([label])).float() if label else None
            tensorized_sample['label'] = label_tensor

        if stereo_pair_id:
            tensorized_sample['stereo_pair_id'] = stereo_pair_id


        
        return tensorized_sample
        

class KeypointsDataset(Dataset):
    """Keypoints dataset

    This is the base version of the dataset that is used to map 3D keypoints to a
    biomass estimate. The label is the weight, and the input is the 3D workd keypoints
    obtained during triangulation

    """

    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform


    def __len__(self):
        return self.df.shape[0]


    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        if self.transform:
            input_sample = {
                'keypoints': row.keypoints,
                'cm': row.camera_metadata,
                'stereo_pair_id': row.id,
            }
            if 'weight' in dict(row).keys():
                input_sample['label'] = row.weight
            sample = self.transform(input_sample)
            return sample

        world_keypoints = row.world_keypoints
        weight = row.weight

        sample = {'kp_input': world_keypoints, 'label': weight, 'stereo_pair_id': row.id}

        return sample

def main():

    # load some data
    rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))
    query = """
        select * from keypoint_annotations 
        where pen_id=61 and captured_at >= '2019-09-13' and captured_at <= '2019-09-20'
        and keypoints is not null
        order by captured_at
        limit 10;
    """
    df = rds_access_utils.extract_from_database(query)

    # initialize data transforms so that we can run inference with neural network
    normalize_centered_2D_transform = NormalizeCentered2D()
    to_tensor_transform = ToTensor()


    # create dataset and dataloader
    dataset = KeypointsDataset(df, transform=transforms.Compose([
                                            NormalizeCentered2D(lo=0.3, hi=2.0, jitter=10, center=True),
                                            ToTensor()
                                         ]))

    dataloader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=1)
    for X_batch in dataloader:
        print(X_batch['kp_input'])
        print(X_batch['stereo_pair_id'])
        break


        
if __name__ == '__main__':
    main()





In [ ]:
sample = {
        'keypoints': keypoints,
        'cm': cm,
        'stereo_pair_id': row.id,
        'single_point_inference': True
    }

In [ ]:
keypoints, cm, stereo_pair_id, label = \
            sample['keypoints'], sample['cm'], sample.get('stereo_pair_id'), sample.get('label')
left_keypoints_list = keypoints['leftCrop']
right_keypoints_list = keypoints['rightCrop']
left_kps = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in left_keypoints_list}
right_kps = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in right_keypoints_list}

In [ ]:
def flip_center_kps(left_kps, right_kps):

    x_min_l = min([kp[0] for kp in left_kps.values()])
    x_max_l = max([kp[0] for kp in left_kps.values()])
    x_mid_l = np.mean([x_min_l, x_max_l])

    y_min_l = min([kp[1] for kp in left_kps.values()])
    y_max_l = max([kp[1] for kp in left_kps.values()])
    y_mid_l = np.mean([y_min_l, y_max_l])

    x_min_r = min([kp[0] for kp in right_kps.values()])
    x_max_r = max([kp[0] for kp in right_kps.values()])
    x_mid_r = np.mean([x_min_r, x_max_r])

    y_min_r = min([kp[1] for kp in right_kps.values()])
    y_max_r = max([kp[1] for kp in right_kps.values()])
    y_mid_r = np.mean([y_min_r, y_max_r])
    
    print(x_mid_r)
    print(y_mid_r)

    fc_left_kps, fc_right_kps = {}, {}
    flip_factor = 1 if left_kps['UPPER_LIP'][0] > left_kps['TAIL_NOTCH'][0] else -1
    for bp in BODY_PARTS:
        left_kp, right_kp = left_kps[bp], right_kps[bp]
        if flip_factor > 0:
            fc_left_kp = np.array([left_kp[0] - x_mid_l, left_kp[1] - y_mid_l])
            fc_right_kp = np.array([right_kp[0] - x_mid_l, right_kp[1] - y_mid_l])
        else:
            fc_right_kp = np.array([x_mid_r - left_kp[0], left_kp[1] - y_mid_r])
            fc_left_kp = np.array([x_mid_r - right_kp[0], right_kp[1] - y_mid_r])
        fc_left_kps[bp] = fc_left_kp
        fc_right_kps[bp] = fc_right_kp

    return fc_left_kps, fc_right_kps

def _rotate_cc(p, theta):
    R = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)]
    ])

    rotated_kp = np.dot(R, p)
    return rotated_kp


def rotate_kps(left_kps, right_kps):
    upper_lip_x, upper_lip_y = left_kps['UPPER_LIP']
    theta = np.arctan(upper_lip_y / upper_lip_x)
    print(theta)
    r_left_kps, r_right_kps = {}, {}
    for bp in BODY_PARTS:
        rotated_kp = _rotate_cc(left_kps[bp], -theta)
        r_left_kps[bp] = rotated_kp
        disp = abs(left_kps[bp][0] - right_kps[bp][0])
        r_right_kps[bp] = np.array([rotated_kp[0] - disp, rotated_kp[1]])

    return r_left_kps, r_right_kps

In [ ]:
flip_center_kps(left_kps, right_kps)

In [ ]:
a, b = flip_center_kps(left_kps, right_kps)

In [ ]:
left_kps, right_kps = rotate_kps(a, b)

In [ ]:
left_kps_list = []
right_kps_list = []
for bp in BODY_PARTS:
    l_item = {
        'keypointType': bp,
        'xFrame': left_kps[bp][0] + cm['pixelCountWidth'] / 2.0,
        'yFrame': left_kps[bp][1] + cm['pixelCountHeight'] / 2.0
    }

    r_item = {
        'keypointType': bp,
        'xFrame': right_kps[bp][0] + cm['pixelCountWidth'] / 2.0,
        'yFrame': right_kps[bp][1] + cm['pixelCountHeight'] / 2.0
    }

    left_kps_list.append(l_item)
    right_kps_list.append(r_item)

modified_kps = {
    'leftCrop': left_kps_list,
    'rightCrop': right_kps_list
}





In [ ]:
pixel2world(modified_kps['leftCrop'], modified_kps['rightCrop'], cm)

In [ ]:
modified_kps